This notebook is a Pytorch practice based on the Youtube channrl [Nicholas Renotte](https://www.youtube.com/watch?v=mozBidd58VQ&list=PLgNJO2hghbmjY-_2OZbm69f9Ga-eIqFui&index=3)

In [18]:
import torch
from torch import nn, save, load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from PIL import Image

In [2]:
train = datasets.MNIST(root="data", download=True, train=True, transform=ToTensor())
dataset = DataLoader(train, 32)
#1,28,28
#Classes: 0-9

In [6]:
class ImageClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(1, 32, (3,3)),
        nn.ReLU(),
        nn.Conv2d(32, 64, (3,3)),
        nn.ReLU(),
        nn.Conv2d(64, 64, (3,3)),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(64*(28-6)*(28-6), 10)

    )

  def forward(self, x):
        return self.model(x)

  

In [7]:
#Instance of neural network, loss, optimizer
clf = ImageClassifier().to('cuda')
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()


In [16]:
#Training flow

if __name__ == "__main__":
  for epoch in range(10):
    for batch in dataset:
      X,y = batch
      X, y = X.to('cuda'), y.to('cuda')
      yhat = clf(X)
      loss = loss_fn(yhat, y)
       
      opt.zero_grad()
      loss.backward()
      opt.step()

    print(f"Epoch:{epoch} loss is {loss.item()}")

  #save the model
  with open('model_state_pt', 'wb') as f:
    save(clf.state_dict(), f)


Epoch:0 loss is 0.00014723371714353561
Epoch:1 loss is 0.0
Epoch:2 loss is 1.4901157641133977e-08
Epoch:3 loss is 0.0
Epoch:4 loss is 0.0
Epoch:5 loss is 0.0
Epoch:6 loss is 0.0
Epoch:7 loss is 0.0
Epoch:8 loss is 0.0
Epoch:9 loss is 0.0


In [21]:
if __name__ == "__main__":

  with open('model_state_pt', 'rb') as f:
    clf.load_state_dict(load(f))

  img = Image.open('/content/drive/MyDrive/Work/Practice/numpy/test_img/img_3.jpg')
  img_tensor = ToTensor()(img).unsqueeze(0).to('cuda')

  print(torch.argmax(clf(img_tensor)))

tensor(9, device='cuda:0')
